In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import networkx as nx
from tqdm import tqdm
import pickle

# Read and Understand Dataset 

***
**Read the `train.txt` file to dataframe**
***

In [2]:
train_data=pd.read_csv("train.txt", delimiter=",", header=None,names=['Neighbours'],index_col=False)

In [3]:
train_data['ID']=train_data['Neighbours'].apply(lambda x: x.split('\t')[0])  # Get the ID

In [4]:
train_data['Neighbours']=train_data['Neighbours'].apply(lambda x: x.split('\t')[1:]) # set up the neighbours

In [5]:
train_data=train_data[["ID","Neighbours"]] # set "ID" as the index

In [7]:
num_neighbours=[] # the number of successors for each source.
for elem in train_data['Neighbours']:
    num_neighbours.append(len(elem))
train_data['Num_neighbours']=num_neighbours

In [8]:
train_data.shape

(20000, 3)

# Transfer Data to Directed Graph and Analysis the Di-Graph

***
**Transfer the dataframe to the graph G=<V,E>**
***

In [9]:
num_source=train_data.shape[0]
sink=train_data.iloc[0,]
len_sink=len(sink)

In [10]:
train_data

,ID,Neighbours,Num_neighbours
0,540762,"[1912140, 1537559, 3091331, 2757277, 3237295, ...",143
1,2129843,"[65840, 3414168, 4523797, 2851163, 4321895, 13...",21
2,3361377,"[955840, 3342058, 1536902, 1850727, 1504632, 1...",764195
3,1199298,"[2300061, 2635670, 2803600, 744722, 881446, 28...",297
4,1392121,"[3845572, 546016, 4361302, 678461, 4294597, 24...",3808
...,...,...,...
19995,585576,"[660302, 3279973, 2094235, 2355188, 1296935, 3...",64
19996,505961,"[3875645, 2148630, 4288909, 4011139, 340232, 1...",539
19997,125824,"[54421, 868022, 385000, 2050130, 3446665, 2040...",45
19998,896087,"[431577, 1007572, 499457, 3642500, 3734728, 28...",219


In [ ]:
# build the directed_graph
diG = nx.DiGraph()
for i in range(num_source):
    source=train_data.iloc[i,0] # The sources
    sinks=train_data.iloc[i,1] # Neighbours
    len_sink=len(sinks)
    for j in range(len_sink):
        sink=sinks[j]
        diG.add_edge(source,sink)
        if sink not in diG.nodes:
            diG.add_node(sink)      

In [17]:
list(diG.edges)[:5]

[('540762', '1912140'),
 ('540762', '1537559'),
 ('540762', '3091331'),
 ('540762', '2757277'),
 ('540762', '3237295')]

In [18]:
print("The Di-graph contains %d nodes and %d edges" %(len(diG.nodes),len(diG.edges)))

The Di-graph contains 4867136 nodes and 23946602 edges


***
Save the graph to `graph.txt`
***

In [ ]:
with open('graph.txt','wb') as file:
    pickle.dump(diG,file)

***
Bulid the adjacent matrix for the `source nodes` to find the positive and neigative samples
***

In [11]:
nodelist=train_data.iloc[:,0]
nodelist.shape

(20000,)

In [ ]:
adj_matrix=nx.to_numpy_matrix(graph,nodelist) # adjacent matrix, only consider the source nodes

In [18]:
adj_matrix.shape

In [ ]:
with open ('adj_matrix.txt', 'wb') as file:
    pickle.dump(adj_matrixe, file)

***
**Gerenrate the positive samples:`edges_pairs` and negative samples `No_edges_pairs`**
***

In [ ]:
# Negative Sample
No_edges_pairs = []

# traverse adjacency matrix
offset = 0
for i in range(adj_Matrix.shape[0]):
    for j in range(offset,adj_Matrix.shape[1]):
        if i != j:
            if adj_Matrix[i,j] == 0 and nx.shortest_path_length(graph,nodelist[i],nodelist[j])==3:
                No_edges_pairs.append([nodelist[i],nodelist[j]])
    offset = offset + 1

In [ ]:
# Positive Samples


In [ ]:
# Combined Samples


In [ ]:
with open('samples.txt','wb') as file:
    pickle.dump(sample,file)

In [ ]:
with open('samples_200000.txt','rb') as file:
    sample_data=pickle.load(file)

In [93]:
sample_data[：5]

,Source,Sink,Label
0,1604753,3794609,1
1,1927211,2402534,1
2,3438576,3589472,1
3,2029671,2363623,1
4,2031305,4169107,1


# Feature Exaction

***
**Vertices features**

Contains: Source_following, Sink_follows, shortest_path
***

## Find the Percentage of source's following --> `Source_following`

In [106]:
source=sample_data['Source']
#out_degree=np.zeros(sample_data.shape[0])
out=[]
for elem in source:
    out.append(graph.out_degree(elem))
maxs=max(out)

In [100]:
out_degree=list(i/maxs for i in out)

In [107]:
out_degree.count(1) #'761793' has the largest 'following' number

9033

## Generate the Percentage of Sink's Follower--> `Sink_follows`

In [119]:
in_=[]
sink=sample_data['Sink']
for elem in sink:
    in_.append(graph.in_degree(elem))
max_num=max(in_)

In [121]:
in_degree=list(i/max_num for i in in_)

In [122]:
in_degree.count(1) # '3361377' has the most followers

19

## The shortest path between source and sink -->`shortest_path`

In [133]:
shortest_path=[]
n=sample_data.shape[0]
source_L=list(source) 
sink_L=list(sink)
for i in tqdm(range(n)):
    lenth=nx.shortest_path_length(graph,source_L[i],sink_L[i])
    shortest_path.append(lenth)

100%|██████████| 200000/200000 [52:30<00:00, 63.49it/s]  


In [137]:
sample_data.head()

,Label,Source,Sink,Source_following,Sink_follows,shortest_path
0,1,1604753,3794609,0.004253,0.000207,1
1,1,1927211,2402534,0.148757,0.009089,1
2,1,3438576,3589472,0.180827,0.001653,1
3,1,2029671,2363623,0.001629,0.008056,1
4,1,2031305,4169107,0.030763,0.024169,1


In [138]:
with open ('data6.txt', 'wb') as file:
    pickle.dump(sample_data, file)

In [ ]:
adj_Matrix=nx.to_numpy_matrix(diG,nodelist)

***
**Feature Exaction: similarity**

Contains: resource allocation index, jaccard coefficient, adamic adar index, preferential attachment, cn_soundarajan_hopcroft, ra_index_soundarajan_hopcroft, within_inter_cluster
***


In [ ]:
unG=graph.to_undirected()
nodelist=list(train_data.iloc[:,0]) # sourse node

In [ ]:
nodelist=list(train_data.iloc[:,0]) # sourse node
rai = nx.resource_allocation_index(unG, nodelist) # resource_allocation_index
jc = nx.jaccard_coefficient(unG, nodelist) # jaccard coefficient
aai = nx.adamic_adar_index(unG, nodelist) # adamic adar index
pa = nx.preferential_attachment(unG, nodelist) # preferential attachment
csj = nx.cn_soundarajan_hopcroft(unG, nodelist) # soundarajan hopcroft
rjsh = nx.ra_index_soundarajan_hopcroft(unG, nodelist) # ra index soundarajan hopcroft
wic = nx.within_inter_cluster(unG, nodelist) #within_inter_cluster

# The Final Dataframe --> `sample_data`

***
**Build the new dataset with the seleted features**
***

In [124]:
sample_data['Source_following']=out_degree
sample_data['Sink_follows']=in_degree
sample_data['shortest_path']=shortest_path

In [126]:
sample_data=sample_data[['Label','Source','Sink','Source_following','Sink_follows','shortest_path']] # put 'Label' to the first column